# PyTrilinos: Epetra Maps & Vectors


# Epetra Maps

* A "map" describes the distribution of global node (or element) indices across all processors
* A "node" is a calculation point, e.g. finite elements, finite differences, etc.
* "global" refers to a node numbering scheme that describes the entire model, i.e. what it would be if it where only run on a single processor
* "local" refers to a on processor node numbering scheme 
* 4 ways to construct a map

```python
Map(numGE, iBase, comm)
Map(numGE, numME, iBase, comm)
Map(numGE, myGEs, iBase, comm)
Map(map)
```


# Epetra Map: Example 1


In [2]:
%%file EpetraMap1.py
#!/usr/bin/env python
from PyTrilinos import Epetra
comm = Epetra.PyComm()
standard_map = Epetra.Map(10, 0, comm)
print("My global indices are: " + str(standard_map.MyGlobalElements()))

Overwriting EpetraMap1.py


In [ ]:
!mpiexec -np 2 python EpetraMap1.py

# Epetra Map: Example 2


In [ ]:
%%file EpetraMap2.py
#!/usr/bin/env python
from PyTrilinos import Epetra
comm = Epetra.PyComm() 

number_of_global_elements = 20 
number_of_elements_on_first_processor = 4

if comm.MyPID() == 0:
    number_of_my_elements = number_of_elements_on_first_processor
else:
    number_of_my_elements = (number_of_global_elements -
                             number_of_elements_on_first_processor) // (comm.NumProc() - 1)
    
standard_map = Epetra.Map(number_of_global_elements, number_of_my_elements, 0, comm)
print("My global indices are: " + str(standard_map.MyGlobalElements()))

In [ ]:
!mpiexec -np 2 python EpetraMap2.py

In [ ]:
!mpiexec -np 3 python EpetraMap2.py

# Epetra Map: Example 3


In [ ]:
%%file EpetraMap3.py
#!/usr/bin/env python
from PyTrilinos import Epetra
comm = Epetra.PyComm()

if comm.MyPID() == 0:
    my_global_elements = [0,2,4]
else:
    my_global_elements = [1,3,5,6]
    
standard_map = Epetra.Map(7, my_global_elements, 0, comm)
print ("My global indices are: " + str(standard_map.MyGlobalElements()))

In [ ]:
!mpiexec -np 2 python EpetraMap3.py

# Epetra Vectors

* Inherits from Numpy's `numpy.ndarray` 
  * Epetra Vectors are Numpy arrays

* Distributed over processors according to the associated `Epetra.Map()`
* Type: `double`
  * See `Epetra.IntVector()` for `int` type 
  
* Several useful constructors

```python
Vector(map, zeroOut=True)
Vector(map, array)
Vector(vector)
```


# Epetra Vector: Example 1


In [ ]:
%%file EpetraVector1.py
#!/usr/bin/env python
from PyTrilinos import Epetra
import numpy as np

comm = Epetra.PyComm()
standard_map = Epetra.Map(10, 0, comm)
x = Epetra.Vector(standard_map)

x[:] = np.arange(standard_map.NumMyElements())

print("My (local, global) indices are: "+ str(np.array([x, standard_map.MyGlobalElements()], dtype=np.int).T))

In [ ]:
!mpiexec -np 2 python EpetraVector1.py

# Epetra Vector: Example 2


In [ ]:
%%file EpetraVector2.py
#!/usr/bin/env python
from PyTrilinos import Epetra 
import numpy as np

comm = Epetra.PyComm()
standard_map = Epetra.Map(10, 0, comm)

x = Epetra.Vector(standard_map) 
y = Epetra.Vector(standard_map)

x[:] = np.arange(standard_map.NumMyElements()) 
y[:] = x + 1

tmp1 = x.Dot(y) 
tmp2 = x.Norm2() 
tmp3 = y.MaxValue()

if comm.MyPID() == 0:
    print("x . y = " + str(tmp1[0]))
    print("The L2 norm of x is = " + str(tmp2[0])) 
    print("The max value of y is = " + str(tmp3[0]))

In [ ]:
!mpiexec -np 2 python EpetraVector2.py

# Epetra Comm MPI-Style Methods

* Several MPI-style methods available
  
  ```python
   Broadcast(numpy.ndarray, int root)
   GatherAll(PyObject obj) -> numpy.ndarray
   SumAll(PyObject obj) -> numpy.ndarray
   MaxAll(PyObject obj) -> numpy.ndarray
   MinAll(PyObject obj) -> numpy.ndarray
   ```
   
* All MPI methods available via `GetMpiComm`
  * Example: `comm.GetMpiComm.Scatter(PyObject obj, root=0)`


# Epetra Comm: Example


In [ ]:
%%file EpetraComm.py
#!/usr/bin/env python
from PyTrilinos import Epetra 
import numpy as np
comm = Epetra.PyComm()
x = np.arange(5)
tmp1 = comm.GatherAll(x)
if comm.MyPID() == 0: print(tmp1)

In [ ]:
!mpiexec -np 2 python EpetraComm.py

# Epetra BlockMap and Epetra Multivector

* Epetra Maps are Epetra BlockMaps
  * With Point Size = 1
  * Most `Map()` methods are defined for `BlockMaps()`

* Epetra Vectors are Epetra MultiVectors
  * With number of dimensions = 1
  * Most `Vector()` methods are defined for `MultiVectors()`


In [3]:
%%javascript
function hideElements(elements, start) {
    for(var i = 0, length = elements.length; i < length;i++) {
        if(i >= start) {
            elements[i].style.display = "none";
        }
    }
}

var prompt_elements = document.getElementsByClassName("prompt");
hideElements(prompt_elements, 0)

<IPython.core.display.Javascript object>